In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import pickle
import logging

from inputs import read_reservoir_attributes, read_reservoir_data
from starfit import Starfit

from lisfloodreservoirs.utils.metrics import KGEmod, compute_performance
from lisfloodreservoirs import Config, read_attributes, read_timeseries

## Configuration 

In [2]:
cfg = Config('Z:/nahaUsers/casadje/datasets/reservoirs/ResOpsUS/results/starfit/config_2var.yml')

### Logger

In [3]:
# create logger
logger = logging.getLogger('simulate-starfit')
logger.setLevel(logging.INFO)
logger.propagate = False
log_format = logging.Formatter('%(asctime)s | %(levelname)s | %(name)s | %(message)s', datefmt='%Y-%m-%d %H:%M:%S')
# log on screen
c_handler = logging.StreamHandler()
c_handler.setFormatter(log_format)
c_handler.setLevel(logging.INFO)
logger.addHandler(c_handler)
# # log file
# log_path = cfg.PATH_CALIB / 'logs'
# log_path.mkdir(exist_ok=True)
# log_file = log_path / '{0:%Y%m%d%H%M}_calibrate_{1}.log'.format(datetime.now(),
#                                                                '_'.join(args.config_file.split('.')[0].split('_')[1:]))
# f_handler = logging.FileHandler(log_file)
# f_handler.setFormatter(log_format)
# f_handler.setLevel(logging.INFO)
# logger.addHandler(f_handler)

logger.info(f'Results will be saved in: {cfg.PATH_CALIB}')

2024-07-31 12:42:01 | INFO | simulate-starfit | Results will be saved in: Z:\nahaUsers\casadje\datasets\reservoirs\ResOpsUS\results\starfit\calibration\bivariate


## Data

### Attributes

In [4]:
# list of reservoirs to be trained
try:
    reservoirs = pd.read_csv(cfg.RESERVOIRS_FILE, header=None).squeeze().tolist()
except IOError as e:
    logger.error(f'Failed to open {cfg.RESERVOIRS_FILE}: {e}')
    raise

# import all tables of attributes
try:
    attributes = read_attributes(cfg.PATH_DATA / 'attributes', reservoirs)
except IOError as e:
    logger.error('Failed to read attribute tables from {0}: {1}'.format(cfg.PATH_DATA / 'attributes', e))
    raise
logger.info(f'{attributes.shape[0]} reservoirs in the attribute tables')

2024-07-31 12:42:02 | INFO | simulate-starfit | 90 reservoirs in the attribute tables


### Time series

In [5]:
# training periods
try:
    with open(cfg.PERIODS_FILE, 'rb') as file:
        periods = pickle.load(file)
except IOError as e:
    logger.error(f'Failed to open {cfg.PERIODS_FILE}: {e}')
    raise

# read time series
try:
    timeseries = read_timeseries(cfg.PATH_DATA / 'time_series' / 'csv',
                                 attributes.index,
                                 periods)
except IOError as e:
    logger.error('Failed to read time series from {0}: {1}'.format(cfg.PATH_DATA / 'time_series' / 'csv', e))
    raise
logger.info(f'{len(timeseries)} reservoirs with timeseries')

2024-07-31 12:42:06 | INFO | simulate-starfit | 90 reservoirs with timeseries


## Class

In [10]:
for grand_id, obs in tqdm(timeseries.items(), desc='simulating reservoir'):
        
    # add day of the year to the observed time series
    obs['doy'] = obs.index.dayofyear   

    # SIMULTE OPTIMIZED RESERVOIR
    
    try:
        
        # load fitted storage model
        # model_storage = pd.read_csv(cfg.PATH_DEF.parent / 'NOR' / f'{grand_id}.csv')
        with open(cfg.PATH_DEF.parent / 'NOR' / f'{grand_id}.pkl', 'rb') as file:
            model_storage = pickle.load(file)
        NOR = pd.DataFrame({
                'flood': model_storage["NOR upper bound"],
                'conservation': model_storage["NOR lower bound"]
            })
        Vtot = model_storage['capacity (MCM)'] * 1e6
        
        # load fitted release model
        with open(cfg.PATH_DEF.parent / 'release' / f'{grand_id}.pkl', 'rb') as file:
            model_release = pickle.load(file)
        avg_inflow = model_release['mean inflow (MCM/wk)'] * 1e6 / (7 * 86400) # m3/s
        Qmin, Qmax = (model_release['constraints'] + 1) * avg_inflow

        # declare reservoir
        res = Starfit(Vtot=Vtot,
                      avg_inflow=avg_inflow,
                      pars_Vf=NOR['flood'],
                      pars_Vc=NOR['conservation'],
                      pars_Qharm=model_release['harmonic parameters'],
                      pars_Qresid=model_release['residual parameters'],
                      Qmin=Qmin,
                      Qmax=Qmax)
        
        # export calibrated parameters
        # with open(cfg.PATH_CALIB / f'{grand_id}_optimal_parameters.yml', 'w') as file:
        #     yaml.dump(res.get_params(), file)
        
        # simulate
        sim = res.simulate(obs.inflow, obs.storage.iloc[0])
        sim.to_csv(cfg.PATH_CALIB / f'{grand_id}_simulation.csv', float_format='%.3f')
        
        logger.info(f'Simulation of the calibrated reservoir {grand_id} successfully finished')
        
    except RuntimeError as e:
        logger.error(f'Calibrated reservoir {grand_id} could not be simulated: {e}')
        continue

    # ANALYSE RESULTS

    # performance
    try:
        performance_cal = compute_performance(obs, sim)
        performance_cal.to_csv(cfg.PATH_CALIB / f'{grand_id}_performance.csv', float_format='%.3f')
        logger.info(f'Performance of reservoir {grand_id} has been computed')
    except IOError as e:
        logger.error(f'The performance of reservoir {grand_id} could not be exported: {e}')

    # scatter plot simulation vs observation
    try:
        res.scatter(sim,
                    obs,
                    norm=False,
                    title=f'grand_id: {grand_id}',
                    save=cfg.PATH_CALIB / f'{grand_id}_scatter.jpg',
                   )
        logger.info(f'Scatter plot of simulation from reservoir {grand_id}')
    except IOError as e:
        logger.error(f'The scatter plot of reservoir {grand_id} could not be generated: {e}')

    # line plot calibration (vs default simulation) vs observation
    try:
        res.lineplot({'starfit': sim},
                     obs, 
                     Vlims=[res.Vtot],
                     Qlims=[res.Qmin, res.Qmax],
                     figsize=(12, 6),
                     save=cfg.PATH_CALIB / f'{grand_id}_line.jpg',
                   )
        logger.info(f'Line plot of simulation from reservoir {grand_id}')
    except IOError as e:
        logger.error(f'The line plot of reservoir {grand_id} could not be generated: {e}')

simulating reservoir:   0%|          | 0/90 [00:00<?, ?it/s]

2024-07-31 12:42:36 | INFO | simulate-starfit | Simulation of the calibrated reservoir 41 successfully finished
2024-07-31 12:42:36 | INFO | simulate-starfit | Performance of reservoir 41 has been computed
2024-07-31 12:42:37 | INFO | simulate-starfit | Scatter plot of simulation from reservoir 41
2024-07-31 12:42:37 | INFO | simulate-starfit | Line plot of simulation from reservoir 41
2024-07-31 12:42:43 | INFO | simulate-starfit | Simulation of the calibrated reservoir 63 successfully finished
2024-07-31 12:42:43 | INFO | simulate-starfit | Performance of reservoir 63 has been computed
2024-07-31 12:42:44 | INFO | simulate-starfit | Scatter plot of simulation from reservoir 63
2024-07-31 12:42:44 | INFO | simulate-starfit | Line plot of simulation from reservoir 63
2024-07-31 12:42:46 | INFO | simulate-starfit | Simulation of the calibrated reservoir 293 successfully finished
2024-07-31 12:42:46 | INFO | simulate-starfit | Performance of reservoir 293 has been computed
2024-07-31 12:

In [7]:
# obs = timeseries[362]

# sim = res.simulate(obs.inflow, obs.storage.iloc[0])
# sim['doy'] = sim.index.dayofweek
# sim = sim.reset_index().merge(res.NOR * res.Vtot, on='doy').merge(res.avg_inflow * (res.Qharm + 1), on='doy').set_index('date')
# sim.sort_index(inplace=True)

In [8]:
# res.lineplot({'starfit': sim},
#                      obs, 
#                      Vlims=[res.Vtot],
#                      Qlims=[res.Qmin, res.Qmax],
#                      figsize=(12, 6),
#                      save=None,
#              xlim=('1996-01-01', '1997-01-01')
#             )

In [9]:
# fig, ax = plt.subplots(nrows=2, figsize=(12, 8), sharex=True)

# sim.loc['1995-10-15':'1995-11-01', ['storage', 'flood']].plot(ax=ax[0]);

# sim.loc['1995-10-15':'1995-11-01', ['inflow', 'outflow', 'release']].plot(ax=ax[1])
# ax[1].axhline(res.Qmax, ls=':', color='k')
# ;